In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import callbacks
from tensorflow.keras import Sequential, layers, regularizers
import numpy as np
from scipy.stats import pearsonr, spearmanr
from tensorflow.keras.callbacks import EarlyStopping
import talib
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.losses import Loss
from tensorflow.keras.utils import register_keras_serializable
import tensorflow.keras.backend as K

In [46]:
path = os.path.join(".", "Data", "sample", "AAPL.csv")

df = pd.read_csv(path)

df.head()

,Dt,Adj Close,Close,Div,Factor,High,Low,Open,Volume
0,2017-01-03,110.9418,116.15,0.0,1.0,116.3300,114.76,115.80,28781865.0
1,2017-01-04,110.8176,116.02,0.0,1.0,116.5100,115.75,115.85,21118116.0
2,2017-01-05,111.3812,116.61,0.0,1.0,116.8642,115.81,115.92,22193587.0
3,2017-01-06,112.6229,117.91,0.0,1.0,118.1600,116.47,116.78,31751900.0
4,2017-01-09,113.6545,118.99,0.0,1.0,119.4300,117.94,117.95,33561948.0


In [47]:
df.drop(columns=['Factor','Div'],inplace=True)

In [53]:
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")  # close enough for rough estimate

tokens = tokenizer.encode(table_text)
print("Token count:", len(tokens))


Token indices sequence length is longer than the specified maximum sequence length for this model (15178 > 1024). Running this sequence through the model will result in indexing errors


Token count: 15178


In [48]:
# Convert to JSON-style text
row_texts = df.apply(lambda row: row.to_dict(), axis=1).to_list()

# Format into readable string
table_text = "\n".join([str(row) for row in row_texts])

table_text

"{'Dt': '2017-01-03', 'Adj Close': 110.9418, 'Close': 116.15, 'High': 116.33, 'Low': 114.76, 'Open': 115.8, 'Volume': 28781865.0}\n{'Dt': '2017-01-04', 'Adj Close': 110.8176, 'Close': 116.02, 'High': 116.51, 'Low': 115.75, 'Open': 115.85, 'Volume': 21118116.0}\n{'Dt': '2017-01-05', 'Adj Close': 111.3812, 'Close': 116.61, 'High': 116.8642, 'Low': 115.81, 'Open': 115.92, 'Volume': 22193587.0}\n{'Dt': '2017-01-06', 'Adj Close': 112.6229, 'Close': 117.91, 'High': 118.16, 'Low': 116.47, 'Open': 116.78, 'Volume': 31751900.0}\n{'Dt': '2017-01-09', 'Adj Close': 113.6545, 'Close': 118.99, 'High': 119.43, 'Low': 117.94, 'Open': 117.95, 'Volume': 33561948.0}\n{'Dt': '2017-01-10', 'Adj Close': 113.7691, 'Close': 119.11, 'High': 119.38, 'Low': 118.3, 'Open': 118.77, 'Volume': 24462051.0}\n{'Dt': '2017-01-11', 'Adj Close': 114.3804, 'Close': 119.75, 'High': 119.93, 'Low': 118.6, 'Open': 118.74, 'Volume': 27588593.0}\n{'Dt': '2017-01-12', 'Adj Close': 113.9028, 'Close': 119.25, 'High': 119.3, 'Low': 

In [52]:
from openai import OpenAI

client = OpenAI(
    api_key="hidden",
    base_url="https://api.groq.com/openai/v1"
)

user_prompt = f"""
Here is stock price data:

{table_text}

Question: What day had the highest adj closing price, and how did the volume on that day compare to the others?
Don't give instructions on Python codes.
"""

response = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
        {"role": "system", "content": "You are a financial data assistant."},
        {"role": "user", "content": user_prompt}
    ],
)

print(response.choices[0].message.content)


To find the day with the highest adj closing price, we need to compare the 'Adj Close' values for all days. 

The highest adj closing price occurred on 2017-12-18 with a value of 171.2318.

The volume on this day was 28831533, which is higher than the average volume but not the highest volume overall. The highest volume occurred on 2017-02-01 with a value of 111985040, which is roughly 3.9 times higher than the volume on 2017-12-18. However, the volume on 2017-12-18 is still one of the higher volumes in the dataset.
